# 构造函数
**构造函数(constructor)**,既初始化方法,命名为 \_\_init\_\_ 。然而,构造函数不同于普通方法的地方在于,将在对象创建后自动调用它们。在Python中,创建构造函数很容易,只需将方法 init 的名称从普通的 init 改为魔法版 \_\_init\_\_即可。

In [1]:
class FooBar1:
    def __init__(self):
        self.somevar = 42

f = FooBar1()
f.somevar

42

In [2]:
class FooBar2:
    def __init__(self,value=12):
        self.value = value

f = FooBar2(23)
f.value

23

**注意**  Python提供了魔法方法 \_\_del\_\_ ,也称作**析构函数(destructor)**。这个方法在对象被销毁(作为垃圾被收集)前被调用,但鉴于你无法知道准确的调用时间,建议尽可能不要使用 \_\_del\_\_ 。

### 重写普通方法和特殊的构造函数
每个类都有一个或多个超类,并从它们那里继承行为。对类 B 的实例调用方法(或访问其属性)时,如果找不到该方法(或属性),将在其超类 A 中查找。

In [3]:
class A:
    def hello(self):
        print("超类A")
class B(A):
    pass

b = B()
b.hello()

超类A


要在子类中添加功能,一种基本方式是添加方法。可以重写超类的某些方法,以定制继承而来的行为。

In [4]:
class C(A):      #C可以重写方法hello
    def hello(self):
        print("类C")
        
c = C()
c.hello()

类C


重写是继承机制的一个重要方面,对构造函数来说尤其重要。构造函数用于初始化新建对象的状态,而对大多数子类来说,除超类的初始化代码外,还需要有自己的初始化代码。虽然所有方法的重写机制都相同,但与重写普通方法相比,重写构造函数时更有可能遇到一个特别的问题:重写构造函数时,必须调用超类(继承的类)的构造函数,否则可能无法正确地初始化对象。

In [5]:
class Bird:
    def __init__(self):
        self.hungry = True
    def eat(self):
        if self.hungry:
            print('吧唧吧唧。。。')
            self.hungry = False
        else:
            print('不饿')
            
b = Bird()
b.eat()
b.eat()

吧唧吧唧。。。
不饿


In [6]:
class SongBird(Bird):   #鸟类的子类
    def __init__(self):
        self.sound = 'Squawk!'
    def sing(self):
        print(self.sound)

        
s = SongBird()
s.sing()
s.eat()   #SongBird 是 Bird 的子类,继承了方法 eat ,但如果调用它,将发生异常

Squawk!


AttributeError: 'SongBird' object has no attribute 'hungry'

出现这个异常的原因是在 SongBird 中重写了构造函数,但新的构造函数没有包含任何初始化属性 hungry 的代码。要消除这种错误, SongBird 的构造函数必须调用其超类( Bird )的构造函数,以确保基本的初始化得以执行。为此,有两种方法:调用未关联的超类构造函数,以及使用函数 super 。如下所示：

### 调用未关联的超类构造函数（了解即可）
此方法用于解决历史遗留问题，在较新的Python版本中,应使用函数super，有所了解即可。此外，这种方法也极具启迪意义,淋漓尽致地说明了关联方法和未关联方法之间的差别。

In [7]:
class SongBird2(Bird):
    def __init__(self):
        Bird.__init__(self)
        self.sound = 'Squawk!'
    def sing(self):
        print(self.sound)
        
s = SongBird2()
s.eat()

吧唧吧唧。。。


**为什么要这样做？** <br>对实例调用方法时,方法的参数 self 将自动关联到实例(称为关联的方法)。然而,如果通过类调用方法(如 Bird.\_\_init\_\_ ),就没有实例与其相关联。在这种情况下,你可随便设置参数 self 。这样的方法称为**未关联**的。通过将这个未关联方法的 self 参数设置为当前实例,将使用超类的构造函数来初始化SongBird2 对象。这意味着将设置其属性 hungry 。

### 使用函数 super
此函数只适用于新式类，调用这个函数时,将当前类和当前实例作为参数。对其返回的对象调用方法时,调用的将是超类(而不是当前类)的方法。因此,在SongBird 的构造函数中,可不使用 Bird ,而是使用 super(SongBird, self) 。另外,可像通常那样(也就是像调用关联的方法那样)调用方法 \_\_init\_\_ 。在Python 3中调用函数 super 时,可不提供任何参数(通常也应该这样做)。按照该方法修改后的类如下所示：

In [8]:
class SongBird3(Bird):
    def __init__(self):
        super().__init__()
        self.sound = 'Squawk!'
    def sing(self):
        print(self.sound)
        
s = SongBird3()
s.eat()     #新版本与旧版本等效

吧唧吧唧。。。


<center>**函数super()的优点**<center/> <br>
    
&emsp;&emsp;相比于直接对超类调用未关联方法,使用函数 super 更直观,但这并非其唯一的优点。实际上,函数 super 很聪明,因此即便有多个超类,也只需调用函数 super 一次(条件是所有超类的构造函数也使用函数 super )。另外,对于使用旧式类时处理起来很棘手的问题(如两个超类从同一个类派生而来),在使用新式类和函数 super 时将自动得到处理。你无需知道函数 super 的内部工作原理,但必须知道的是,使用函数 super 比调用超类的未关联构造函数(或其他方法)要好得多。 <br>

&emsp;&emsp;函数 super 返回的是一个 super 对象,这个对象将负责为你执行方法解析。当你访问它的属性时,它将在所有的超类(以及超类的超类,等等)中查找,直到找到指定的属性或引发 AttributeError 异常。

# 元素访问

### 基本的序列和映射协议
序列和映射基本上是**元素(item)**的集合,要实现它们的基本行为(协议),不可变对象需要实现2个方法,而可变对象需要实现4个。 <br>

* \_\_len\_\_(self) :这个方法应返回集合包含的项数,对序列来说为元素个数,对映射来说为键-值对数。如果 \_\_len\_\_ 返回零(且没有实现覆盖这种行为的 \_\_nonzero\_\_ ),对象在布尔上下文中将被视为假(就像空的列表、元组、字符串和字典一样)。
* \_\_getitem\_\_(self, key) :这个方法应返回与指定键相关联的值。对序列来说,键应该是0~n-1的整数(也可以是负数,这将在后面说明),其中n为序列的长度。对映射来说,键可以是任何类型。
* \_\_setitem\_\_(self, key, value) :这个方法应以与键相关联的方式存储值,以便以后能够使用 \_\_getitem\_\_ 来获取。当然,仅当对象可变时才需要实现这个方法。
* \_\_delitem\_\_(self, key) :这个方法在对对象的组成部分使用 \_\_del\_\_ 语句时被调用,应删除与 key 相关联的值。同样,仅当对象可变(且允许其项被删除)时,才需要实现这个方法。
<br>
对于这些方法,还有一些额外的要求: <br>
* 对于序列,如果键为负整数,应从末尾往前数。换而言之, x[-n] 应与 x[len(x)-n] 等效。
* 如果键的类型不合适(如对序列使用字符串键),可能引发 TypeError 异常。
* 对于序列,如果索引的类型是正确的,但不在允许的范围内,应引发 IndexError 异常。  <br> <br>
关于更复杂的接口和使用的抽象基类( Sequence ),请参阅有关模块 collections 的文档，链接如下： <br>
https://docs.python.org/3/library/collections.html

In [9]:
def check_index(key):
    if not isinstance(key,int):raise TypeError   #如果输入的不是整数将引发TypeError异常
    if key<0:raise IndexError   #如果输入的是负数将引发IndexError异常

class ArithmeticSequence:
    def __init__(self,start=0,step=1):
        self.start = start
        self.step = step
        self.changed = {}    #一个字典，包含被修改的值
    def __getitem__(self,key):
        check_index(key)
        try: return self.changed[key]       #是否修改过
        except KeyError:
            return self.start + key * self.step    #如果没有修改过就计算元素的值
    def __setitem__(self,key,value):
        check_index(key)
        self.changed[key] = value
        
a = ArithmeticSequence(1,2)
print(a[4])
a[5] = 3
print(a[5])

9
3


### 从 list 、 dict 和 str 派生
如果要实现一种行为类似于内置列表的序列类型，可直接继承list，示例如下：

In [10]:
#一个带访问计数器的列表
class CounterList(list):
    def __init__(self, *args):
        super().__init__(*args)
        self.counter = 0
    def __getitem__(self, index):
        self.counter += 1
        return super(CounterList, self).__getitem__(index) 
    
c = CounterList((1,2,3,4,5))
print(c[3])
c.counter

4


1

CounterList类依赖于其超类（list）的行为。CounterList没有重写的方法（如append、extend、index等）都可直接使用。在两个被重写的方法中，使用super来调用超类的相应方法，并添加了必要的行为：初始化属性counter（在\_\_init\_\_中）和更新属性counter（在\_\_getitem\_\_中）。 <br>
**注意** 重写\_\_getitem\_\_并不能保证一定会捕捉用户的访问操作，因为还有其他访问列表内容的方式，如通过方法pop。

# 其他魔法方法
Python中还有大量的魔法方法，其详细的介绍在下面的链接中： <br>
https://docs.python.org/3/reference/datamodel.html

# 特性
通过存取方法定义的属性通常称为**特性（property）**。在Python中，有两种创建特性的机制，其中较新的为函数property，它只能用于新式类。另一种机制是使用魔法方法来实现特性。

### 函数property

In [11]:
class Rectangle:
    def __init__ (self):
        self.width = 0
        self.height = 0
    def set_size(self, size):
        self.width, self.height = size
    def get_size(self):
        return self.width, self.height
    size = property(get_size, set_size) #函数property使用起来非常简单，只需添加这行代码即可
    
r = Rectangle()
r.width = 50
r.height = 100
print("size:",r.size)
r.size = 30,40
print("width:",r.width)
print("height:",r.height)

size: (50, 100)
width: 30
height: 40


在这个新版的Rectangle中，通过调用函数property并将存取方法作为参数（**获取方法在前，设置方法在后**）创建了一个特性，然后将名称size关联到这个特性。这样，你就能以同样的方式对待width、height和size，而无需关心它们是如何实现的。size依然受制于函数gei_size和set_size，但看起来依然和普通属性一样。 <br>
在调用函数property时，如果没有指定任何参数，创建的特性将既不可读也不可写。如果只指定一个参数（获取方法），创建的特性将是只读的。第三个参数是可选的，指定用于删除属性的方法（这个方法不接受任何参数）。第四个参数也是可选的，指定一个文档字符串。这些参数分别名为fget、fset、fdel和doc。 <br>

<center>**函数property的工作原理**<center/>
    
property其实并不是函数，而是一个类。它的实例包含一些魔法方法，而所有的魔法都是由这些方法完成的。这些魔法方法为\_\_get\_\_、\_\_set\_\_和\_\_delete\_\_，它们一道定义了所谓的描述符协议。只要对象实现了这些方法中的任何一个，它就是一个描述符。描述符的独特之处在于其访问方式。例如，读取属性（具体来说，是在实例中访问类中定义的属性）时，如果它关联的是一个实现了\_\_get\_\_的对象，将不会返回这个对象，而是调用方法\_\_get\_\_并将其结果返回。实际上，这是隐藏在特性、关联的方法、静态方法和类方法以及super后面的机制。  <br>
**关于描述符的文档链接如下** <br>
https://docs.python.org/3/howto/descriptor.html

### 静态方法和类方法
静态方法和类方法是这样创建的：将它们分别包装在staticmethod和classmethod类的对象中。静态方法的定义中没有参数self，可直接通过类来调用。类方法的定义中包含类似于self的参数，通常被命名为cls。对于类方法，也可通过对象直接调用，但参数cls将自动关联到类。示例如下：

In [12]:
class MyClass:
    def smeth():
        print("这是一个静态方法!")
    smeth = staticmethod(smeth)
    def cmeth(cls):
        print("这是一个类方法：",cls)
    cmeth = classmethod(cmeth)
    
MyClass.smeth()   #无需实例化类
MyClass.cmeth()

这是一个静态方法!
这是一个类方法： <class '__main__.MyClass'>


像这样手工包装和替换方法有点繁琐。应当使用**装饰器**来包装方法。（实际上，装饰器可用于包装任何可调用的对象，并且可用于特性，方法和函数。）可指定一个或多个装饰器，为此可在方法（或函数）前面使用运算符@列出这些装饰器（指定了多个装饰器时，应用的顺序与列出的顺序相反）。

In [13]:
class MyClass2:
    @staticmethod
    def smeth():
        print("这是一个静态方法！")
        
    @classmethod
    def cmeth(cls):
        print("这是一个类方法:",cls)
        
MyClass2.smeth()    #无需实例化类
MyClass2.cmeth()

这是一个静态方法！
这是一个类方法: <class '__main__.MyClass2'>


装饰器可用于工厂函数 <br>
相关参考文档链接如下：<br>
http://hanxiaomax.github.io/python/python-docorator-note/   <br>
https://www.zhihu.com/question/31265857

### \_\_getattr\_\_、\_\_setattr\_\_等方法
可以拦截对对象属性的所有访问企图，其用途之一是在旧式类中实现特性（在旧式类中，函数property的行为可能不符合预期）。要在属性被访问时执行一段代码，必须使用一些魔法方法。下面的四个魔法方法提供了你需要的所有功能（在旧式类中，只需使用后面三个）。 <br>
* \_\_getattribute\_\_(self, name)：在属性被访问时自动调用（只适用于新式类）。
* \_\_getattr\_\_(self, name)：在属性被访问而对象没有这样的属性时自动调用。
* \_\_setattr\_\_(self, name, value)：试图给属性赋值时自动调用。
* \_\_delattr\_\_(self, name)：试图删除属性时自动调用。
<br>
相比函数property，这些魔法方法使用起来要棘手些（从某种程度上说，效率也更低），但它们很有用，因为你可在这些方法中编写处理多个特性的代码。然而，在可能的情况下，还是使用函数property吧。使用魔法方法的示例如下：

In [14]:
class Rectangle:
    def __init__ (self):
        self.width = 0
        self.height = 0
    def __setattr__(self, name, value):
        if name == 'size':
            self.width, self.height = value
        else:
            self. __dict__[name] = value
    def __getattr__(self, name):
        if name == 'size':
            return self.width, self.height
        else:
            raise AttributeError() 

r = Rectangle()
r.size = 10,20
print(r.size)
r.test = 1,2
print(r.test)

(10, 20)
(1, 2)


* 即便涉及的属性不是size，也将调用方法\_\_setattr\_\_。因此这个方法必须考虑如下两种情形：如果涉及的属性为size，就执行与以前一样的操作；否则就使用魔法属性\_\_dict\_\_。\_\_dict\_\_属性是一个字典，其中包含所有的实例属性。之所以使用它而不是执行常规属性赋值，是因为旨在避免再次调用\_\_setattr\_\_，进而导致无限循环。
* 仅当没有找到指定的属性时，才会调用方法\_\_getattr\_\_。这意味着如果指定的名称不是size，这个方法将引发AttributeError异常。这在要让类能够正确地支持hasattr和getattr等内置函数时很重要。如果指定的名称为size，就使用前一个实现中的表达式。
<br><br>
**注意** <br>编写方法\_\_setattr\_\_时需要避开无限循环陷阱，编写\_\_getattribute\_\_时亦如此。由于它拦截对所有属性的访问（在新式类中），因此将拦截对\_\_dict\_\_的访问！在\_\_getattribute\_\_中访问当前实例的属性时，唯一安全的方式是使用超类的方法\_\_getattribute\_\_（使用super）。

# 迭代器

### 迭代器协议
**迭代（iterate）**意味着重复多次，就像循环那样。可以迭代所有实现了方法\_\_iter\_\_的对象。 <br>
方法\_\_iter\_\_返回一个迭代器，它是包含方法\_\_next\_\_的对象，而调用这个方法时可不提供任何参数。当你调用方法\_\_next\_\_时，迭代器应返回其下一个值。如果迭代器没有可供返回的值，应引发StopIteration异常。你还可使用内置的便利函数next，在这种情况下，next(it)与it.\_\_next\_\_()等效。

In [15]:
class Fibs:   #实现斐波那契数列的迭代器
    def __init__(self):
        self.a = 0
        self.b = 1
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a
    def __iter__(self):    #方法__iter__返回迭代器本身
        return self 

fibs = Fibs()
for i in fibs:
    if i > 20:
        print(i)  #找出第一个大于20的数
        break

21


**注意**  更正规的定义是，实现了方法\_\_iter\_\_的对象是**可迭代的**，而实现了方法\_\_next\_\_的对象是**迭代器**。  <br>
通过对可迭代对象调用内置函数iter，可获得一个迭代器。还可使用它从函数或其他可调用对象创建可迭代对象。

In [16]:
it = iter([1,2,3,4,5,6,7])
print(next(it))
print(next(it))

1
2


### 从迭代器创建序列
除了对迭代器和可迭代对象进行迭代（通常这样做）之外，还可将它们转换为序列。在可以使用序列的情况下，大多也可使用迭代器或可迭代对象（诸如索引和切片等操作除外）。一个例子是使用构造函数list显式地将迭代器转换为列表。

In [17]:
class Test:
    value=0
    def __next__(self):
        self.value+=1
        if self.value > 10 :raise StopIteration
        return self.value
    def __iter__(self):
        return self
    
t = Test()
list(t)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# 生成器
生成器是一种使用普通函数语法定义的迭代器。

### 创建生成器
创建一个将嵌套列表展开的函数。

In [18]:
def flatten(nested):
    for sublist in nested:
        for element in sublist:
            yield element           #包含yield语句的函数都被称为生成器

生成器的行为与普通函数截然不同。差别在于，生成器不是使用return返回一个值，而是可以生成多个值，每次一个。每次使用yield生成一个值后，函数都将冻结，即在此停止执行，等待被重新唤醒。被重新唤醒后，函数将从停止的地方开始继续执行。

In [19]:
nested = [[1, 2], [3, 4], [5]]

for num in flatten(nested):      #为使用所有的值，可对生成器进行迭代
    print(num) 

1
2
3
4
5


In [20]:
list(flatten(nested))

[1, 2, 3, 4, 5]

这里，最难理解的就是生成器和函数的执行流程不一样。函数是顺序执行，遇到return语句或者最后一行函数语句就返回。而生成器在每次调用next()的时候执行，遇到yield语句返回，再次执行时从上次返回的yield语句处继续执行。

In [21]:
def odd():
    print("第一步")
    yield 1
    print("第二步")
    yield 2
    
o = odd()

In [22]:
next(o)

第一步


1

In [23]:
next(o)

第二步


2

In [24]:
next(o)

StopIteration: 

**生成器推导**（也叫**生成器表达式**）其工作原理与列表推导相同，但不是创建一个列表（即不立即执行循环），而是返回一个生成器，让你能够逐步执行计算。

In [25]:
g = ((i+2)**2 for i in range(2,30))    #与列表推导不同的是，这里使用的是圆括号
next(g)

16

在像这样的简单情形下，还不如使用列表推导；但如果要包装可迭代对象（可能生成大量的值），使用列表推导将立即实例化一个列表，从而丧失迭代的优势。另一个好处是，直接在一对既有的圆括号内（如在函数调用中）使用生成器推导时，无需再添加一对圆括号。换而言之，可编写下面这样非常漂亮的代码：

In [26]:
sum(i**2 for i in range(10))

285

### 递归式生成器
前一节设计的生成器只能处理两层的嵌套列表，这是使用两个for循环来实现的。如果要处理任意层嵌套的列表，则可以使用递归式生成器

In [27]:
def flatten2(nested):
    try:
        for sublist in nested:
            for element in flatten2(sublist):
                yield element
    except TypeError:
        yield nested 
        
list(flatten2([[[1], 2], 3, 4, [5, [6, 7]], 8]))

[1, 2, 3, 4, 5, 6, 7, 8]

**注意**  这个解决方案存在一个问题。如果nested是字符串或类似于字符串的对象，它就属于序列，因此不会引发TypeError异常。在函数flatten中，不应该对类似于字符串的对象进行迭代，主要原因有两个。首先，你想将类似于字符串的对象视为原子值，而不是应该展开的序列。其次，对这样的对象进行迭代会导致无穷递归，因为字符串的第一个元素是一个长度为1的字符串，而长度为1的字符串的第一个元素是字符串本身！

In [28]:
def flatten3(nested):
    try:
        try: nested + ''       #不迭代类似于字符串的对象
        except TypeError: pass
        else: raise TypeError
        for sublist in nested:
            for element in flatten3(sublist):
                yield element
    except TypeError:
        yield nested
        
list(flatten3(['foo', ['bar', ['baz']]]))

['foo', 'bar', 'baz']

如果表达式nested + ''引发了TypeError异常，就忽略这种异常；如果没有引发TypeError异常，内部try语句中的else子句将引发TypeError异常，这样将在外部的excpet子句中原封不动地生成类似于字符串的对象。

### 通用生成器
生成器是包含关键字yield的函数，但被调用时不会执行函数体内的代码，而是返回一个迭代器。每次请求值时，都将执行生成器的代码，直到遇到yield或return。yield意味着应生成一个值，而return意味着生成器应停止执行（即不再生成值；仅当在生成器调用return时，才能不提供任何参数）。  <br>
生成器由两个单独的部分组成：**生成器的函数**和**生成器的迭代器**。生成器的函数是由def语句定义的，其中包含yield。生成器的迭代器是这个函数返回的结果。

In [29]:
def simple_generator():
    yield 1
    
simple_generator

<function __main__.simple_generator>

In [30]:
simple_generator()

<generator object simple_generator at 0x7f9ed6ef8308>

### 生成器的方法
在生成器开始运行后，可使用生成器和外部之间的通信渠道向它提供值。这个通信渠道包含如下两个端点：  <br>
* **外部世界**：外部世界可访问生成器的方法send，这个方法类似于next，但接受一个参数（要发送的“消息”，可以是任何对象）。
* **生成器**：在挂起的生成器内部，yield可能用作**表达式**而不是**语句**。换而言之，当生成器重新运行时，yield返回一个值——通过send从外部世界发送的值。如果使用的是next，yield将返回None。<br>

**注意**  仅当生成器被挂起（即遇到第一个yield）后，使用send（而不是next）才有意义。要在此之前向生成器提供信息，可使用生成器的函数的参数。如果一定要在生成器刚启动时对其调用方法send，可向它传递参数None。

In [31]:
def repeater(value):
    while True:
        new = (yield value)
        if new is not None: value = new 

r = repeater(40)
print(next(r))
r.send("hello-world")

40


'hello-world'

生成器还包含另外两个方法：  <br>
**方法throw**：用于在生成器中（yield表达式处）引发异常，调用时可提供一个异常类型、一个可选值和一个traceback对象。  <br>
**方法close**：用于停止生成器，调用时无需提供任何参数。方法close（ 由Python垃圾收集器在需要时调用）也是基于异常的：在yield处引发GeneratorExit异常。因此如果要在生成器中提供一些清理代码，可将yield放在一条try/finally语句中。如果愿意，也可捕获GeneratorExit异常，但随后必须重新引发它（可能在清理后）、引发其他异常或直接返回。对生成器调用close后，再试图从它那里获取值将导致RuntimeError异常。

In [32]:
def MyGenerator():
    value = 1
    while True:
        try:
            yield value
            value += 1
        except:
            value=1

m=MyGenerator()
print(next(m))
print(next(m))
m.throw(Exception,"这是一个测试！")

1
2


1

In [33]:
def MyGenerator2():
    try:
        yield 1
    except GeneratorExit:
        print("生成器被销毁！")
            
m = MyGenerator2()
print(next(m))
m.close()

1
生成器被销毁！


Exception ignored in: <generator object MyGenerator at 0x7f9ed6ef8830>
RuntimeError: generator ignored GeneratorExit


有关生成器的方法以及它们是如何将生成器变成简单协同程序（coroutine）的详细信息，参考以下链接： <br>
https://www.python.org/dev/peps/pep-0342/

# 八皇后问题

In [34]:
import random

def conflict(state, nextX):   #冲突检测
    nextY = len(state)       #nextX，nextY：下一个皇后的水平和垂直坐标
    for i in range(nextY):
        if abs(state[i] - nextX) in (0, nextY - i):  #下一个皇后和当前皇后的水平距离为0（在同一列）或与它们的垂直距离相等（位于一条对角线上）这个表达式就为真；否则为假。
            return True
    return False



def queens(num=8, state=()):    #num为皇后的总数，state为一个元组存储已放好皇后的位置
    for pos in range(num):
        if not conflict(state, pos):
            if len(state) == num-1:    #如果还有一个皇后没有放好
                yield (pos,)
            else:
                for result in queens(num, state + (pos,)):
                    yield (pos,) + result
                
def prettyprint(solution):
    def line(pos, length=len(solution)):
        return '. ' * (pos) + 'X ' + '. ' * (length-pos-1)
    for pos in solution:
        print(line(pos)) 
        
prettyprint(random.choice(list(queens())))

X . . . . . . . 
. . . . . X . . 
. . . . . . . X 
. . X . . . . . 
. . . . . . X . 
. . . X . . . . 
. X . . . . . . 
. . . . X . . . 
